In [195]:
qualitatives_attributes_col_idx <- c(3,5,6,8,9,10)
discretized_quantitatives_attributes_col_idx <- c(3,5)
quantitatives_attributes_col_idx <- c(1,2,4)
target_variables_idx <- 7
datasets_directory <- "../datasets/"
dataset_filename <- "HR_prediction-all.csv"
output_folder <- "output"
working_directory <- "C:/Users/huniv/jnotebook/data-mining/62-62_Data_Mining/tp/tp02" 

In [196]:
# Print all column names with their indices
for (i in seq_along(colnames(myData))) {
	cat("Index:", i, "-> Column Name:", colnames(myData)[i], "\n")
}


Index: 1 -> Column Name: satisfaction_level 
Index: 2 -> Column Name: last_evaluation 
Index: 3 -> Column Name: number_project 
Index: 4 -> Column Name: average_montly_hours 
Index: 5 -> Column Name: time_spend_company 
Index: 6 -> Column Name: Work_accident 
Index: 7 -> Column Name: left 
Index: 8 -> Column Name: promotion_last_5years 
Index: 9 -> Column Name: department 
Index: 10 -> Column Name: salary 


In [197]:
#mettre le paquet Naive Bayes à disposition
if (!requireNamespace("e1071", quietly = TRUE)) { 
  install.packages("e1071")
}

#for latex parsing of equations
if (!requireNamespace("latex2exp", quietly = TRUE)) { 
  install.packages("latex2exp")
}

# Check if the package is installed
if (!require("colorspace", quietly = TRUE)) { 
  install.packages("colorspace")
}

library(e1071)
library(latex2exp)
library(colorspace)

In [198]:
library(arules)
# Function to create an output directory if it doesn't exist
ensure_output_directory <- function(wd, folder_name) {
	output_dir <- file.path(wd, folder_name)
	if (!dir.exists(output_dir)) {
		dir.create(output_dir, recursive = TRUE)
	}
	return(output_dir)
}

In [199]:
setwd(working_directory)
#read data
# avec colClasses, nous contrôlons explicitement les types des différentes variables, 
# les valeurs qui nous intéressent le plus sont 
# factor: variables qualitatives
# numeric: pour les variables quantitatives
myData <- read.table(
  "../datasets/HR_prediction-all.csv",
  header = TRUE,
  sep = ",",
  colClasses = c(
    "NULL",         # Id - Remove   
    "numeric",      # satisfaction_level
    "numeric",      # last_evaluation
    "factor",       # number_project
    "integer",      # average_montly_hours
    "factor",       # time_spend_company
    "factor",       # Work_accident (categorical: 0 or 1)
    "factor",       # left (target variable: 0 or 1)
    "factor",       # promotion_last_5years (categorical: 0 or 1)
    "factor",       # department
    "factor"        # salary
  )
)

In [200]:
head(myData)

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,department,salary
,<dbl>,<dbl>,<fct>,<int>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,0.83,0.80,4,158,4,0,0,0,marketing,medium
2,1.00,0.39,2,210,5,0,0,0,sales,low
3,0.97,0.91,3,246,2,0,0,0,accounting,low
4,0.68,0.65,3,173,3,1,0,0,marketing,high
5,0.77,0.55,3,225,3,0,0,0,sales,medium
6,0.51,0.71,2,180,3,0,0,0,product_mng,low


In [201]:
evaluate_classification_model <- function(
  data, 
  target_col_idx, 
  n_iter, 
  shouldBeReproductible = TRUE, 
  model_function, 
  predict_function
) {
    # Ensures reproductibility
    if(shouldBeReproductible) {
        set.seed(42)
    }
    
    # Extract the target variable name
    target_name <- colnames(data)[target_col_idx]
    
    # Initialize a vector to store accuracies
    accuracies <- numeric(n_iter)
  
	# Initialize a variable to store the trained model
	final_model <- NULL
    
    for (i in 1:n_iter) {
        # Split data into training and testing sets
        train_idx <- sample(1:nrow(data), size = floor(2/3 * nrow(data)))
        train_data <- data[train_idx, ]
        test_data <- data[-train_idx, ]
        
        # Train a Naive Bayes on data.
        # The parameter:
        #   formula=left~.
        # sets the target/class variable to be the left and use as predictive variables all the others
        
        # Build dynamic formula
        formula <- as.formula(paste(target_name, "~ ."))
        
        # Train the model using the input model function
        model <- model_function(formula, train_data)
        
        # Predict on test data using the input predict function
        predictions <- predict_function(model, test_data)
        
        # Calculate accuracy
        accuracy <- sum(predictions == test_data[[target_name]]) / nrow(test_data)
        accuracies[i] <- accuracy
        
        cat("Accuracy for iteration", i, ":", round(accuracy, 4), "\n")
	    
	    # Update the final model
	    final_model <- model
    }
    
    # Report the average accuracy
    avg_accuracy <- mean(accuracies)
    cat("\nAverage Accuracy over", n_iter, "splits:", round(avg_accuracy, 4), "\n")
	
	# Return the average accuracy and the last trained model
	return(list(
		average_accuracy = avg_accuracy,
		trained_model = final_model
	))
}

In [202]:
# Function to train the default classifier
trainDefaultClassifier <- function(train_data, target_idx) {
	# Extract the target column
	target_column <- train_data[,target_idx]
	
	# Find the majority class using a frequency table
	majority_class <- names(which.max(table(target_column)))
	
	# Return the majority class
	return(majority_class)
}

# Function to predict using the default classifier
predictDefaultClassifier <- function(test_data, majority_class) {
	# Create a vector of predictions with the majority class
	# Basically repeat the majority class for the number of rows in the test data
	predictions <- rep(majority_class, nrow(test_data))
	
	# Return the predictions
	return(predictions)
}

In [203]:
# Split the data into train and test sets
set.seed(42)

train_idx <- sample(1:nrow(myData), size = floor(2/3 * nrow(myData)))
train_data <- myData[train_idx, ]
test_data <- myData[-train_idx, ]

# Train the default classifier to find the majority class
majority_class <- trainDefaultClassifier(train_data, target_variables_idx)
cat("")
cat("Majority class identified:", majority_class, "\n")

# Use the default classifier to make predictions on the test set
predictions <- predictDefaultClassifier(test_data, majority_class)

# Evaluate the accuracy of the default classifier
accuracy <- sum(predictions == test_data[[target_variables_idx]]) / nrow(test_data)
cat("Default Classifier Accuracy:", round(accuracy, 4), "\n")

Majority class identified: 0 
Default Classifier Accuracy: 0.7618 


In [204]:
# Single split
set.seed(1)
train_idx <- sample(1:nrow(myData), size = floor(2/3 * nrow(myData)))
train_data <- myData[train_idx, ]
test_data <- myData[-train_idx, ]
model <- naiveBayes(left ~ ., data = train_data)
# Calculate training accuracy
train_predictions <- predict(model, train_data)
train_accuracy <- sum(train_predictions == train_data[["left"]]) / nrow(train_data)
cat("Training Accuracy:", round(train_accuracy, 4), "\n")
# Test accuracy
test_accuracy <- sum(predictions == test_data[["left"]]) / nrow(test_data)
cat(paste("Test Accuracy:", round(test_accuracy, 4), "\n"))

model

Training Accuracy: 0.8608 
Test Accuracy: 0.7642 



Naive Bayes Classifier for Discrete Predictors

Call:
naiveBayes.default(x = X, y = Y, laplace = laplace)

A-priori probabilities:
Y
        0         1 
0.7607261 0.2392739 

Conditional probabilities:
   satisfaction_level
Y        [,1]      [,2]
  0 0.6673634 0.2171659
  1 0.4404514 0.2643050

   last_evaluation
Y        [,1]      [,2]
  0 0.7161014 0.1631721
  1 0.7173229 0.1976392

   number_project
Y            2          3          4          5          6          7
  0 0.07533031 0.34450799 0.35062118 0.18181818 0.04772234 0.00000000
  1 0.44263323 0.01880878 0.10219436 0.18181818 0.18307210 0.07147335

   average_montly_hours
Y       [,1]     [,2]
  0 198.8878 45.64708
  1 206.1718 60.87805

   time_spend_company
Y           10          2          3          4          5          6
  0 0.01853678 0.28081246 0.42535989 0.14415303 0.05580753 0.04397555
  1 0.00000000 0.01065831 0.44890282 0.24514107 0.24137931 0.05391850
   time_spend_company
Y            7          8
  0 0.017

In [205]:
results <- evaluate_classification_model(
	myData,
	target_variables_idx,
	5,
	TRUE,
	naiveBayes,
	predict
)

# Extract results
avg_accuracy <- results$average_accuracy
trained_model <- results$trained_model

# Print the average accuracy
cat("Final Average Accuracy:", avg_accuracy, "\n")

# Inspect the final trained model
print(trained_model)

Accuracy for iteration 1 : 0.8605 
Accuracy for iteration 2 : 0.8692 
Accuracy for iteration 3 : 0.8587 
Accuracy for iteration 4 : 0.8599 
Accuracy for iteration 5 : 0.8542 

Average Accuracy over 5 splits: 0.8605 
Final Average Accuracy: 0.8605279 

Naive Bayes Classifier for Discrete Predictors

Call:
naiveBayes.default(x = X, y = Y, laplace = laplace)

A-priori probabilities:
Y
       0        1 
0.759976 0.240024 

Conditional probabilities:
   satisfaction_level
Y        [,1]      [,2]
  0 0.6680083 0.2156065
  1 0.4400875 0.2643933

   last_evaluation
Y        [,1]      [,2]
  0 0.7181958 0.1629576
  1 0.7199313 0.1989800

   number_project
Y            2          3          4          5          6          7
  0 0.07165417 0.34958547 0.35136202 0.17962890 0.04776944 0.00000000
  1 0.43687500 0.01875000 0.11250000 0.17687500 0.18875000 0.06625000

   average_montly_hours
Y       [,1]     [,2]
  0 198.4820 45.83999
  1 207.1481 60.86994

   time_spend_company
Y           10      

In [206]:
# Function to plot and save probability distributions
plot_and_save_probabilities <- function(output_dir, attribute_name, P_target, P_attribute_given_target, target_name, discretized = FALSE) {
	
	# Create the full path for the PDF file
	file_suffix <- ifelse(discretized, "_discretized", "")
	full_pdf_path <- file.path(output_dir, paste0(attribute_name, file_suffix, ".pdf"))
	
	# Open a PDF to save the plots
	pdf(file = full_pdf_path, width = 12, height = 6)
	
	# Set up a 1-row, 2-column layout for the plots
	par(mfrow = c(1, 2), mar = c(7, 4, 4, 2) + 0.1)
	
	# Visualize the probability distribution P(target)
	barplot(P_target,
	        main = paste("Probability Distribution P(", target_name, ")"),
	        xlab = paste("Category of ", target_name),
	        ylab = "Probability")
	
	
	
	# Visualize the conditional probability P(variable | target)
	barplot(P_attribute_given_target,
	        beside = FALSE,
	        main = paste0("Conditional Probability P(", attribute_name, " | ", target_name, ")"),
	        xlab = paste0("Category of ", attribute_name),
	        ylab = "Conditional Probability",
	        legend.text = rownames(P_attribute_given_target),
	        args.legend = list(x = "topright", inset = c(-0.1, 0), bty = "n"))
	
	# Close the PDF device
	dev.off()
	
}

# Function to analyze qualitative attributes
plot_probabilities_from_model_result <- function(wd, folder_name, model, data, variable_idx, target_idx) {
	
	# Ensure output directory exists
	output_dir <- ensure_output_directory(wd, folder_name)
	
	# Calculate the probability distribution P(variable)
	target_name <- colnames(data)[target_idx]
	P_target <- model$apriori
	print(P_target)  # Display the probability distribution
	
	# Calculate the conditional probability P(variable|target)
	# Example : Observing employe who had a work accident among the employe who left the company 
	# P (work_accident = 1 | left=1)
	attribute_name <- colnames(data)[variable_idx]
	P_attribute_given_target <- model$tables[[attribute_name]]
	# Display the conditional probability
	print(P_attribute_given_target)
	
	# Plot and save the probabilities
	plot_and_save_probabilities(output_dir, attribute_name, P_target, P_attribute_given_target, target_name)
}


In [207]:
for (i in qualitatives_attributes_col_idx) {
	plot_probabilities_from_model_result(
			wd = working_directory,
			folder_name = output_folder,
			model = trained_model,
			data = myData,
			variable_idx = i,
			target_idx = target_variables_idx
	)
}


Y
   0    1 
5066 1600 
   number_project
Y            2          3          4          5          6          7
  0 0.07165417 0.34958547 0.35136202 0.17962890 0.04776944 0.00000000
  1 0.43687500 0.01875000 0.11250000 0.17687500 0.18875000 0.06625000
Y
   0    1 
5066 1600 
   time_spend_company
Y           10          2          3          4          5          6
  0 0.01993683 0.27615476 0.43031978 0.14271615 0.05783656 0.04382156
  1 0.00000000 0.01312500 0.44562500 0.25312500 0.22750000 0.06062500
   time_spend_company
Y            7          8
  0 0.01638373 0.01283064
  1 0.00000000 0.00000000
Y
   0    1 
5066 1600 
   Work_accident
Y           0         1
  0 0.8247138 0.1752862
  1 0.9518750 0.0481250
Y
   0    1 
5066 1600 
   promotion_last_5years
Y            0          1
  0 0.97572049 0.02427951
  1 0.99375000 0.00625000
Y
   0    1 
5066 1600 
   department
Y           IT      RandD accounting         hr management  marketing
  0 0.08685353 0.05704698 0.05033557 0.04599